In [24]:
# -*- coding: utf-8 -*-
"""
Yelp Fusion API code sample.

This program demonstrates the capability of the Yelp Fusion API
by using the Search API to query for businesses by a search term and location,
and the Business API to query additional information about the top result
from the search query.


This program requires the Python requests library, which you can install via:
`pip install -r requirements.txt`.
"""

from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib
import pandas as pd 



# This client code can run on Python 2.x or 3.x.  Your imports can be
# simpler if you only need one of those.
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode


# Yelp Fusion no longer uses OAuth as of December 7, 2017.
# You no longer need to provide Client ID to fetch Data
# It now uses private keys to authenticate requests (API Key)
# You can find it on
# https://www.yelp.com/developers/v3/manage_app

# merci de saisir l' API_KEY
API_KEY= XXXXXX 

# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
DEFAULT_TERM = 'restaurants'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 50


def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.

    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.

    Returns:
        dict: The JSON response from the request.

    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)
    
    return response.json()


def request_reviews(host, path, api_key, url_params=None):
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    response = requests.request('GET', url, headers=headers, params=url_params)
    
    return response.json()


def search(api_key, term, location):
    """Query the Search API by a search term and location.

    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.

    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


def get_business(api_key, business_id):
    """Query the Business API by a business ID.

    Args:
        business_id (str): The ID of the business to query.

    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)


def get_reviews(api_key, business_id):
    """Query the Business API by a business ID.

    Args:
        business_id (str): The ID of the business to query.

    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id+'/reviews'

    return request_reviews(API_HOST, business_path, api_key)

Cet API permet de récupérer les reviews et les photos pour chaque restaurant. Elle fournit en sortie deux fichiers en csv :
    - fichier Yelp_text.csv  : contenant les champs business_id, reviews, stars
    - fichier Yelp_photo.csv : contenant les champs business_id, photos

In [25]:
def query_api(term, location):
    """Queries the API by the input values from the user.

    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, term, location)
    
    businesses = response.get('businesses')
    
    print ('Nombre de restaurants :', len(businesses))

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return
    
  
    liste_business_id = []
    liste_photos_business_id = []
    liste_rating = []
    liste_rev_business_id = []
    liste_photos= [] 
    liste_reviews = []
    for i in range(len(businesses)) :
        business_id = businesses[i]['id']
        response = get_business(API_KEY, business_id)
        liste_business_id.append(business_id)
        photos=response['photos']
        for l in range(len(photos)) :
            liste_photos.append(photos[l])
            liste_photos_business_id.append(business_id)
        
        liste_provisoire =[]
        M=get_reviews(API_KEY, business_id)
        for j in range (len(M['reviews'])) :
            reviews=M['reviews'][j]['text']
            rating=M['reviews'][j]['rating']
            liste_reviews.append(reviews)
            liste_rating.append(rating)
            liste_rev_business_id.append(business_id)

    DF_text=pd.DataFrame({'business_id': liste_rev_business_id, 'reviews':liste_reviews, 'stars': liste_rating})
    DF_photo=pd.DataFrame({'business_id': liste_photos_business_id, 'photos':liste_photos})

    DF_text.to_csv('Yelp_text.csv',index=False)
    DF_photo.to_csv('Yelp_photo.csv',index=False) 

In [26]:
query_api("restaurants", "San Francisco")

Querying https://api.yelp.com/v3/businesses/search ...
Nombre de restaurants : 50
Querying https://api.yelp.com/v3/businesses/MNhRp6GhBiwCgGycyI6Mdw ...
Querying https://api.yelp.com/v3/businesses/f-m7-hyFzkf0HSEeQ2s-9A ...
Querying https://api.yelp.com/v3/businesses/AtFGQccBpgEZzsy9V1330Q ...
Querying https://api.yelp.com/v3/businesses/8jW9a4WCnMsQSyT0x1MI8Q ...
Querying https://api.yelp.com/v3/businesses/XQLmEdXoMzOpffwoFaBtaQ ...
Querying https://api.yelp.com/v3/businesses/gqVl3RprESEqkIPeJH0yOg ...
Querying https://api.yelp.com/v3/businesses/lJAGnYzku5zSaLnQ_T6_GQ ...
Querying https://api.yelp.com/v3/businesses/_1pt3zjz2Om9QC3OUSuTtA ...
Querying https://api.yelp.com/v3/businesses/-DrR38H1Abk0wCyu9XOLug ...
Querying https://api.yelp.com/v3/businesses/8kck3-K4zYKTJbJko0JlXQ ...
Querying https://api.yelp.com/v3/businesses/Xg-FyjVKAN70LO4u4Z1ozg ...
Querying https://api.yelp.com/v3/businesses/i09UMzccKgyLwGYKDVP28w ...
Querying https://api.yelp.com/v3/businesses/jdfO0RGI1Wspn6sCR3OY7Q